In [1]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# # Initialize the WebDriver
# driver = webdriver.Chrome()  # or webdriver.Firefox(), or any other WebDriver you're using

# # Open the webpage
# driver.get("https://unabated.com/nba/props")
# driver.execute_script("document.body.style.zoom='30%'")
# time.sleep(2)
# # Find the div element using XPath
# parent_div = driver.find_element(By.CSS_SELECTOR, ".ag-center-cols-container")
# # div_element = parent_div.find_elements(By.TAG_NAME, "div")
# div_element = parent_div.find_elements(By.CSS_SELECTOR, '.ag-center-cols-container > div')
# # child_divs = viewport_div.find_elements_by_css_selector('.ag-center-cols-container > div')
# # print(div_element.text)
# # print(div_element.get_attribute('row-id'))

# for a in div_element:
#     print(a.text)
#     temp = a.get_attribute('row-id')
#     if temp is not None:
#         print(a.get_attribute('row-id'))

# # Do something with the div_element, for example, get its text


# # Close the WebDriver
# driver.quit()

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import re
from datetime import datetime


def extract_text(s):
    # Define a regular expression pattern to match the desired components
    pattern = r'o(\d+\.\d+)\s*(-?\d+)\nu(\d+\.\d+)\s*([-+]\d+)'

    # Use re.findall to find all occurrences of the pattern in the string
    matches = re.findall(pattern, s)

    # Extract the matched groups from each occurrence
    # results = [(float(match[0]), int(match[1]), match[2], int(match[3])) for match in matches]
    if len(matches) == 0:
        return []
    # print(matches[0])
    return [float(matches[0][0]),int(matches[0][1]),float(matches[0][2]),int(matches[0][3])]

def oavgc(row):
    o = {}
    o2 = {}
    for c in df.columns:
        if "o" in row[c]:
            results = extract_text(row[c])
            if len(results) != 4:
                continue
            if results[0] in o:
                o[results[0]].append(results[1])
            else:
                o[results[0]] = [results[1]]
    for p in o.keys():
        o2[p] = round(sum(o[p]) / len(o[p]),1)
    return o2

def uavgc(row):
    o = {}
    o2 = {}
    for c in df.columns:
        if "u" in row[c]:
            results = extract_text(row[c])
            if len(results) != 4:
                continue
            if results[0] in o:
                o[results[0]].append(results[3])
            else:
                o[results[0]] = [results[3]]
    for p in o.keys():
        o2[p] = round(sum(o[p]) / len(o[p]),1)
    return o2

def cleanpd(s):
    o = s.replace("  ", " ,").replace("\no", "\n,o").replace("\n ", "\n, ")
    return(o.split(","))

# Assuming you have set up your WebDriver, such as ChromeDriver
driver = webdriver.Chrome()

# Navigate to your webpage
driver.get("https://unabated.com/nba/props")
driver.execute_script("document.body.style.zoom='30%'")
time.sleep(0.3)

# Find names of sportsbooks used
col_ids = []
id2sbnames = {}
sbln = driver.find_elements(By.CSS_SELECTOR, ".ag-header-container > div:nth-child(1) > div")
for sb in sbln:
    col_ids.append(sb.get_attribute("col-id"))
# print(col_ids)
sbtitle = sb.find_elements(By.XPATH, "//div/div[@style and @title]")
for n,s in enumerate(sbtitle):
    tit = s.get_attribute("title")
    if "Real" in tit or "Low" in tit:
        titsho = tit.split(" \n")[0]
        id2sbnames[col_ids[n-1]] = titsho
        # print("tit",tit)
    
# print(id2sbnames)

# Find the first div with class .ag-center-cols-viewport
viewport_div = driver.find_element(By.CSS_SELECTOR,'.ag-center-cols-viewport')
name_div = driver.find_element(By.CSS_SELECTOR,'.ag-pinned-left-cols-container')

# Keep track of the number of child divs found
last_child_count = 0
player_data = {}
id2name = {}
visited_pages = []
pagebutton = driver.find_element(By.CSS_SELECTOR, "div.dropdown:nth-child(2) > button:nth-child(1)")
driver.execute_script("arguments[0].click();", pagebutton)
dropdowndiv = driver.find_element(By.CSS_SELECTOR, "div.dropdown:nth-child(2) > div:nth-child(2)")
pages = dropdowndiv.find_elements(By.CSS_SELECTOR, "div.dropdown:nth-child(2) > div:nth-child(2) > button")
findpage = True

while True:
    prevpages = visited_pages
    # print("WHILE STARTED", visited_pages)
    for page in pages:
        attempts = 0
        parsed = False
        while(attempts < 5 and not parsed):
            try:
                page_name = page.text
                parsed = True
                break
            except:
                attempts += 1
        if page_name not in visited_pages and findpage:
            print("Scanning page:", page_name)
            break
            
    visited_pages.append(page_name)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", page)
    # print("CLICKED")
    time.sleep(3)

    while True:

        ## FIND ALL ROWS AND CELLS
        child_divs = viewport_div.find_elements(By.CSS_SELECTOR, '.ag-center-cols-container > div')
        id_child = name_div.find_elements(By.CSS_SELECTOR, ".ag-pinned-left-cols-container > div")
        for index, row in enumerate(child_divs):
            try:
                cells = row.find_elements(By.CSS_SELECTOR, "div.ag-cell.ag-cell-not-inline-editing.ag-cell-auto-height.ag-cell-value")
                player_id = row.get_attribute('row-id')
                player_data[player_id] = {}
                for cell in cells:
                    col_id = cell.get_attribute("col-id")
                    player_data[player_id][col_id] = cell.text
            except:
                pass
        

        ## GET PLAYER NAME
        for index, div in enumerate(id_child, start=1):
            id2name[div.get_attribute('row-id')] = div.text
                
        ## CHECK FOR BREAK CONDITIONS
        if len(player_data.keys()) == last_child_count:
            break
        last_child_count = len(player_data.keys())

        ## SCROLL
        time.sleep(0.3)
        actions = ActionChains(driver)
        body = child_divs[-1]
        actions.move_to_element(body)
        try:
            actions.click_and_hold().move_by_offset(0, 100).release().perform()
        except:
            pass

    labeled_player_data = {}

    for key in player_data:
        pname = id2name[key].split("\nEdit")[0].replace("\n", " ")
        labeled_player_data[pname] = {}
        for key2 in player_data[key]:
            labeled_player_data[pname][id2sbnames[key2]] = player_data[key][key2]

    df = pd.DataFrame(labeled_player_data).T
    df['Over'] = df.apply(oavgc, axis=1)
    df['Under'] = df.apply(uavgc, axis=1)
    now = datetime.now()
    date_time_str = now.strftime("Date %Y-%m-%d.  Hour %H")
    df.to_csv(page_name + " " + date_time_str + '.csv')

    ## TIME TO FIND A NEW PAGE TO CLICK
    pagebutton = driver.find_element(By.CSS_SELECTOR, "div.dropdown:nth-child(2) > button:nth-child(1)")
    driver.execute_script("arguments[0].click();", pagebutton)
    time.sleep(0.5)
    attempts = 0
    parsed = False
    while(attempts < 20 and not parsed):
        try:
            dropdowndiv = driver.find_element(By.XPATH, "//div[@class='ml-2 dropdown-menu show']")
            parsed = True
            break
        except:
            attempts += 1
    
    pages = dropdowndiv.find_elements(By.TAG_NAME, "button")
    print("Passed pages")
    for page in pages:
        attempts = 0
        parsed = False
        while(attempts < 5 and not parsed):
            try:
                page_name = page.text
                parsed = True
                break
            except:
                attempts += 1
        if page_name not in visited_pages:
            print("Found new page:", page_name)
            break
    
    if len(visited_pages) == len(pages) or prevpages == len(visited_pages):
        print("BREAKING", len(visited_pages), len(pages))
        print("All pages processed")
        break
    
        
# Loop to keep scrolling down until no more new child divs are loaded


# Finally, remember to close the browser window when done
driver.quit()


3pts Made
AssistsSIM
Blocks
PointsSIM
Pts+AstSIM
Pts+RebSIM
Pts+Reb+AstSIM
ReboundsSIM
Reb+AstSIM
Steals
Stl+Blk
Turnovers
_________________
Found new page: 3pts Made
PARSED
Passed pages
Found new page: AssistsSIM
Found new page: AssistsSIM
PARSED
Passed pages
Found new page: Blocks
Found new page: Blocks
PARSED
Passed pages
Found new page: PointsSIM
Found new page: PointsSIM
PARSED
Passed pages
Found new page: Pts+AstSIM
Found new page: Pts+AstSIM
PARSED
Passed pages
Found new page: Pts+RebSIM
Found new page: Pts+RebSIM
PARSED
Passed pages
Found new page: Pts+Reb+AstSIM
Found new page: Pts+Reb+AstSIM
PARSED
Passed pages
Found new page: ReboundsSIM
Found new page: ReboundsSIM
PARSED
Passed pages
Found new page: Reb+AstSIM
Found new page: Reb+AstSIM
PARSED
Passed pages
Found new page: Steals
Found new page: Steals
PARSED
Passed pages
Found new page: Stl+Blk
Found new page: Stl+Blk
PARSED
Passed pages
Found new page: Turnovers
Found new page: Turnovers
PARSED
Passed pages
BREAKING 12 12


In [81]:
labeled_player_data = {}

for key in player_data:
    pname = id2name[key].split("\nEdit")[0].replace("\n", " ")
    labeled_player_data[pname] = {}
    for key2 in player_data[key]:
        labeled_player_data[pname][id2sbnames[key2]] = player_data[key][key2]

display(pd.DataFrame(labeled_player_data))

,2/15 8:30 PM Bojan Bogdanovic(G-F) @ MIL,2/15 8:30 PM Jaren Jackson(F-C) MEM vs. MIL,2/15 8:30 PM Luke Kennard(G-F) MEM vs. MIL,2/15 8:30 PM Damian Lillard(G) MIL @ MEM,2/15 8:30 PM Malik Beasley(G) MIL @ MEM,2/15 8:30 PM Giannis Antetokounmpo MIL @ MEM,2/15 8:30 PM Brook Lopez(C) MIL @ MEM,2/15 8:30 PM Santi Aldama(F) MEM vs. MIL,2/15 8:30 PM Vince Williams(G-F) MEM vs. MIL,2/15 9:00 PM Klay Thompson(G) GSW @ UTA,...,2/15 9:00 PM Collin Sexton(G) UTA vs. GSW,2/15 9:00 PM Keyonte George(G) UTA vs. GSW,2/15 10:00 PM Rudy Gobert(C) MIN @ POR,2/15 10:00 PM Jaden McDaniels(F) MIN @ POR,2/15 10:00 PM Anthony Edwards(G) MIN @ POR,2/15 10:00 PM Karl-Anthony Towns MIN @ POR,2/15 10:00 PM Mike Conley(G) MIN @ POR,2/15 10:00 PM Anfernee Simons(G) POR vs. MIN,2/15 10:00 PM Deandre Ayton(C) POR vs. MIN,2/15 10:00 PM Jerami Grant(F) POR vs. MIN
Bookmaker,,,,,,,,,,,...,,,,,,,,,,
Sharp Book Price,,,,,,,,,,,...,,,,,,,,,,
Sporttrade,,,,,,,,,,,...,,,,,,,,,,
Bet365,,,,o24.5 -115\nu24.5 -115,o11.5 -125\nu11.5 -105,o30.5 -110\nu30.5 -120,o11.5 -105\nu11.5 -125,,,o16.5 -115\nu16.5 -115,...,o18.5 -115\nu18.5 -115,o13.5 -110\nu13.5 -120,o14.5 -140\nu14.5 +110,o9.5 -115\nu9.5 -115,o26.5 -115\nu26.5 -115,o21.5 -110\nu21.5 -120,o8.5 -125\nu8.5 -105,,,
DraftKings Pick6,,,,,,,,,,,...,,,,,,,,,,
FanDuel,,o23.5 -115\nu23.5 -111,o12.5 -120\nu12.5 -106,o23.5 -125\nu23.5 -102,o11.5 -111\nu11.5 -115,o29.5 -113\nu29.5 -115,o11.5 -104\nu11.5 -122,o11.5 -108\nu11.5 -118,o12.5 -106\nu12.5 -120,o16.5 -118\nu16.5 -108,...,o18.5 -122\nu18.5 -104,o13.5 -108\nu13.5 -118,o15.5 -113\nu15.5 -113,,o26.5 -111\nu26.5 -115,,,o22.5 -108\nu22.5 -118,o14.5 +100\nu14.5 -128,o19.5 -120\nu19.5 -106
DraftKings,,o22.5 -130\nu22.5 +100,,o24.5 -110\nu24.5 -120,o11.5 -130\nu11.5 +100,o30.5 -110\nu30.5 -120,o11.5 -105\nu11.5 -125,,o11.5 -140\nu11.5 +110,o16.5 -110\nu16.5 -120,...,o18.5 -130\nu18.5 +100,o13.5 -110\nu13.5 -120,o14.5 -135\nu14.5 +105,o9.5 -120\nu9.5 -110,o26.5 -115\nu26.5 -115,o21.5 -110\nu21.5 -120,o8.5 -125\nu8.5 -105,o22.5 -120\nu22.5 -110,o14.5 +100\nu14.5 -130,o19.5 -135\nu19.5 +105
BetMGM,,,,o24.5 -111\nu24.5 -120,o11.5 -128\nu11.5 -105,o30.5 -110\nu30.5 -125,o11.5 -105\nu11.5 -128,,,,...,o18.5 -118\nu18.5 -111,o13.5 -105\nu13.5 -125,o14.5 -128\nu14.5 -105,o9.5 -120\nu9.5 -110,o26.5 -118\nu26.5 -111,o22.5 -120\nu22.5 -110,,o22.5 -105\nu22.5 -128,o14.5 +100\nu14.5 -133,o19.5 -120\nu19.5 -110
Caesars,,o22.5 -114\nu22.5 -120,,o24.5 -114\nu24.5 -120,o11.5 -123\nu11.5 -111,o30.5 -109\nu30.5 -125,o11.5 -120\nu11.5 -114,,o11.5 -128\nu11.5 -106,o16.5 -114\nu16.5 -120,...,o18.5 -115\nu18.5 -119,o13.5 -108\nu13.5 -127,o14.5 -132\nu14.5 -104,o9.5 -120\nu9.5 -114,o26.5 -123\nu26.5 -111,o21.5 -115\nu21.5 -119,o8.5 -128\nu8.5 -106,o22.5 -106\nu22.5 -128,o14.5 -106\nu14.5 -128,o19.5 -118\nu19.5 -118
PointsBet,,,,o24.5 -115\nu24.5 -115,o11.5 -125\nu11.5 -105,o30.5 -110\nu30.5 -120,o11.5 -105\nu11.5 -125,,,o16.5 -110\nu16.5 -120,...,o18.5 -125\nu18.5 -105,o13.5 -110\nu13.5 -120,,,,,,,,


In [78]:
labeled_player_data = {}

for key in player_data:
    labeled_player_data[id2name[key]] = {}
    for key2 in player_data[key]:
        try:
            labeled_player_data[id2name[key]][id2sbnames[key2]] = player_data[key][key2]
        except:
            labeled_player_data[id2name[key]][key2] = player_data[key][key2]
labeled_player_data

{'2/15\n8:30 PM\nBojan Bogdanovic(G-F)\n@\nMIL\nEdit\nProjections\nStart Trial': {'Bookmaker': '  ',
  'Sharp Book Price': '  ',
  'Sporttrade': '  ',
  'Bet365': '  ',
  'DraftKings Pick6': '  ',
  'FanDuel': '  ',
  'DraftKings': '  ',
  'BetMGM': '  ',
  'Caesars': '  ',
  'PointsBet': '  ',
  'BetOnline': '  ',
  'Bovada': '  ',
  'Unibet': '  ',
  'Sugarhouse': '  ',
  '888sports': '  ',
  'BetRivers': '  ',
  'SportsBetting': '  ',
  'Four Winds': '  ',
  'Parx': '  '},
 '2/15\n8:30 PM\nJaren Jackson(F-C)\nMEM\nvs.\nMIL\nEdit\nProjections\nStart Trial': {'Bookmaker': '  ',
  'Sharp Book Price': '  ',
  'Sporttrade': '  ',
  'Bet365': '  ',
  'DraftKings Pick6': '  ',
  'FanDuel': 'o23.5 -115\nu23.5 -111',
  'DraftKings': 'o22.5 -130\nu22.5 +100',
  'BetMGM': '  ',
  'Caesars': 'o22.5 -114\nu22.5 -120',
  'PointsBet': '  ',
  'BetOnline': '  ',
  'Bovada': '  ',
  'Unibet': 'o23.5 -109\nu23.5 -122',
  'Sugarhouse': 'o23.5 -109\nu23.5 -122',
  '888sports': '  ',
  'BetRivers': 'o23

In [65]:
df = pd.DataFrame(player_data)
df
player_data

{'1357-58297': {'8': '  ',
  '7': '  ',
  '67': '  ',
  '78': '  ',
  '84': '  ',
  '2': '  ',
  '1': '  ',
  '4': '  ',
  '20': '  ',
  '3': '  ',
  '9': '  ',
  '10': '  ',
  '31': '  ',
  '27': '  ',
  '13': '  ',
  '17': '  ',
  '54': '  ',
  '22': '  ',
  '25': '  '},
 '42485-58297': {'8': '  ',
  '7': '  ',
  '67': '  ',
  '78': '  ',
  '84': '  ',
  '2': 'o23.5 -115\nu23.5 -111',
  '1': 'o22.5 -130\nu22.5 +100',
  '4': '  ',
  '20': 'o22.5 -114\nu22.5 -120',
  '3': '  ',
  '9': '  ',
  '10': '  ',
  '31': 'o23.5 -110\nu23.5 -121',
  '27': 'o23.5 -110\nu23.5 -121',
  '13': '  ',
  '17': 'o23.5 -110\nu23.5 -121',
  '54': '  ',
  '22': 'o23.5 -110\nu23.5 -121',
  '25': 'o23.5 -110\nu23.5 -121'},
 '42575-58297': {'8': '  ',
  '7': '  ',
  '67': '  ',
  '78': '  ',
  '84': '  ',
  '2': 'o12.5 -120\nu12.5 -106',
  '1': '  ',
  '4': '  ',
  '20': '  ',
  '3': '  ',
  '9': 'o12.5 -120\nu12.5 -108',
  '10': 'o12.5 -120\nu12.5 -108',
  '31': 'o12.5 -130\nu12.5 -103',
  '27': 'o12.5 -130\n

,Over,Under
Luguentz Dort(G)\nOKC\n@\nORL,{9.5: -120.5},{9.5: -108.9}
Josh Giddey(G)\nOKC\n@\nORL,{10.5: -103.6},{10.5: -129.8}
Paolo Banchero(F)\nORL\nvs.\nOKC,{22.5: -113.6},{22.5: -117.4}
Markelle Fultz(G)\nORL\nvs.\nOKC,{},{}
Shai Gilgeous-Alexander\nOKC\n@\nORL,{31.5: -114.5},{31.5: -116.1}
Jalen Suggs(G)\nORL\nvs.\nOKC,{10.5: -126.4},{10.5: -68.7}
Franz Wagner(F)\nORL\nvs.\nOKC,"{21.5: -122.7, 22.5: -107.0}","{21.5: -86.4, 22.5: -121.0}"
Wendell Carter(C)\nORL\nvs.\nOKC,{10.5: -132.7},{10.5: -33.0}
Jalen Williams(G-F)\nOKC\n@\nORL,{18.5: -119.2},{18.5: -112.2}
Chet Holmgren(C-F)\nOKC\n@\nORL,"{15.5: -104.5, 14.5: -145.0}","{15.5: -127.8, 14.5: 115.0}"
